In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Multivariate LR

In [4]:
class Mul_LR(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1) # input, output
    
    def forward(self, x):
        return self.linear(x)

In [5]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = Mul_LR()

# optimizer 
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20 Cost: 31667.599609
Epoch    1/20 Cost: 9926.266602
Epoch    2/20 Cost: 3111.513916
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042496
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762


## Minibatch (Dataset, DataLoader)

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

dataset = CustomDataset()        

dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True, 
    )

In [8]:
# 모델 초기화
model = Mul_LR()

# optimizer 
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples

        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 로그 출력
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20 Cost: 17413.818359
Epoch    0/20 Cost: 7685.123047
Epoch    0/20 Cost: 1045.925659
Epoch    1/20 Cost: 740.765747
Epoch    1/20 Cost: 173.894516
Epoch    1/20 Cost: 88.368492
Epoch    2/20 Cost: 5.412563
Epoch    2/20 Cost: 31.563520
Epoch    2/20 Cost: 0.446407
Epoch    3/20 Cost: 14.127743
Epoch    3/20 Cost: 4.429361
Epoch    3/20 Cost: 2.755815
Epoch    4/20 Cost: 9.345474
Epoch    4/20 Cost: 1.833703
Epoch    4/20 Cost: 10.696621
Epoch    5/20 Cost: 5.655952
Epoch    5/20 Cost: 9.922928
Epoch    5/20 Cost: 3.206425
Epoch    6/20 Cost: 0.443790
Epoch    6/20 Cost: 6.340835
Epoch    6/20 Cost: 14.476576
Epoch    7/20 Cost: 7.039721
Epoch    7/20 Cost: 3.278203
Epoch    7/20 Cost: 6.541093
Epoch    8/20 Cost: 2.021883
Epoch    8/20 Cost: 5.636727
Epoch    8/20 Cost: 13.346638
Epoch    9/20 Cost: 8.091965
Epoch    9/20 Cost: 3.749084
Epoch    9/20 Cost: 11.110264
Epoch   10/20 Cost: 8.288495
Epoch   10/20 Cost: 7.978492
Epoch   10/20 Cost: 4.410089
Epoch   11/20 Cost: 8.